In [1]:
# elasticsearch 서버 접속 모듈 설치
# pip install elasticsearch

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# elastic 서버 접속 
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
es = Elasticsearch("http://10.10.4.10:9200/")
es.info()

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'node-1',
 'cluster_name': 'mpower',
 'cluster_uuid': 'Nnk0icM8QeWRIRqqTYjiTA',
 'version': {'number': '7.17.3',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff',
  'build_date': '2022-04-19T08:11:19.070913226Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
#---------------------------------------------------------------------------
#쿼리에 대해 일반검색해서 임베딩 검색할 후보군 10개 목록 uids를 얻는 함수
#---------------------------------------------------------------------------
def es_search_uids(es, esindex:str, size:int=10, data=None):
    if data is None: #모든 데이터 조회
        data = {"match_all":{}}
    else:
        data = {"match": data}
        
    body = {
        "size": size,
        "query": data,
        "_source":{"includes": ["rfile_name","rfile_text"]}
    }
    
    print(body)
    response = es.search(index=esindex, body=body)
    
    print(response)
    
    rfilename = []
    count = 0
    docs = []
    
    print(f'\n[es_search_uids]===>')
    
    for hit in response["hits"]["hits"]: 
        tmp = hit["_source"]["rfile_name"]

        # 중복 제거
        if tmp and tmp not in rfilename:
            rfilename.append(tmp)
            doc = {}  #dict 선언
            score = hit["_score"]
            if score > UID_MIN_SCORE:  # 6 이상일때만 스코어 계산
                doc['rfile_name'] = hit["_source"]["rfile_name"]      # contextid 담음
                doc['rfile_text'] = hit["_source"]["rfile_text"]      # text 담음.
                doc['score'] = score
                docs.append(doc)
                print(f'\t{doc}')
                count += 1

    
    uids = []
    for doc in docs:
        uids.append(doc['rfile_name'])

    return uids, docs

In [9]:
# 검색
index_name = 'qaindex'
UID_MIN_SCORE = 0.5
# 쿼리 검색
data = {'rfile_text': '할아버지 상 휴가'}
#data = None

uids, docs = es_search_uids(es=es,esindex=index_name, size=10, data=data)

print(docs)

{'size': 10, 'query': {'match': {'rfile_text': '할아버지 상 휴가'}}, '_source': {'includes': ['rfile_name', 'rfile_text']}}
{'took': 8, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 54, 'relation': 'eq'}, 'max_score': 6.0502, 'hits': [{'_index': 'qaindex', '_type': '_doc', '_id': '39Ero4sB5NLaJBxZ5tNn', '_score': 6.0502, '_source': {'rfile_name': '14_35', 'rfile_text': '취업 규칙\n난임치료휴가\n회사는 사원이 인공수정 또는 체외수정 등 난임치료를 받기 위하여 휴가(이하 ‘난임치료휴가’라 한다)를 청구하는 경우에 연간 3일 이내의 휴가를 주어야 하며, 이 경우 최초 1일은 유급으로 한다. \n다만, 사원이 청구한 시기에 휴가를 주는 것이 정상적인 사업 운영에 중대한 지장을 초래하는 경우에는 사원과 협의하여 그 시기를 변경할 수 있다.\n회사는 난임치료휴가를 이유로 해고, 징계 등 불리한 처우를 하여서는 아니된다.\n난임치료휴가의 신청 방법과 절차 등은 남녀고용평등법 시행령에 따른다.'}}, {'_index': 'qaindex', '_type': '_doc', '_id': 'VNEso4sB5NLaJBxZMtb7', '_score': 6.048465, '_source': {'rfile_name': '44_2', 'rfile_text': '경조사 지원 규정\n경사 지원금\n본인 결혼 : 5일 유급 휴가, 500,000원 축의금 지원\n자녀 결혼 : 1일 유급 휴가, 300,000 원 축의금 지원\n형제자매결혼 : 1일 유급휴가, 100,000 원 축의금 지원\n자녀

/tmp/ipykernel_223443/256284496.py:17: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  response = es.search(index=esindex, body=body)


In [11]:
query1 = '?할아버지 상 휴가'
query = query1[1:]
print(query)

할아버지 상 휴가
